# モジュール読み込み

In [45]:
import MeCab
from collections import defaultdict
import pandas as pd
import MeCab
import ipadic
from collections import Counter
import re
import matplotlib.pyplot as plt
import numpy as np
import japanize_matplotlib

# データ集計

## 基礎データ読み込み

In [46]:
data=pd.read_csv("./朝ポキ出演者 共同編集用 スプレッドシート - ニュースの現場から.csv")
data.head(3)

,omny.fm へのリンク,日付,タイトル,コメント,MC,出演者1,出演者2,出演者3,音源編集,備考・メモ・作業された方のお名前(ハンドルネーム可)
0,https://omny.fm/shows/asahi/9-ny-1,"Thu, 06 Aug 2020 02:06:00 +0000",#1 ニューヨーク 感染爆発、でも9割が対応支持 NY覆う「戦時の空気」,NaN,神田大介,"藤原学思,ニューヨーク支局",NaN,NaN,NaN,いーむ（em_em000）
1,https://omny.fm/shows/asahi/9-ny,"Sat, 08 Aug 2020 02:00:00 +0000",#2 ローマ 奔放イタリア、コロナで見せた従順 マフィアのボスも…,NaN,神田大介,"河原田慎一,ローマ支局",NaN,NaN,NaN,いーむ（em_em000）
2,https://omny.fm/shows/asahi/28bbd9d5-fe0e-4a05...,"Sun, 09 Aug 2020 02:00:00 +0000",#3 モスクワ① 医師が事実を訴えると、警察くるロシア プーチン氏は…,NaN,神田大介,"石橋亮介,モスクワ支局",NaN,NaN,NaN,いーむ（em_em000）


## 関数定義

In [47]:
""" MC出演者がいない場合はNone,いる場合は出演者の所属を削除する関数 """
def speakerTakeOut(speaker):
    if (pd.isna(speaker)):
        return None
    else:
        speakerOnly=str(speaker.split(",")[0])
        return speakerOnly
""" 既存のリストに出演者とタイトルの辞書型を追加する関数 """    
def podDict(lists,title,member):
    dic={"title":title,"performer":member}
    lists.append(dic)
    return lists

""" タイトルを読み込み数以外の名詞を出力する関数 """
def collectWords(titleName):
    m = MeCab.Tagger(ipadic.MECAB_ARGS)
    words_lines = m.parse(titleName).split('\n')
    parsed_words = []
    for  word_line in words_lines:
        parsed_words.append(re.split('[\t,]', word_line))
    #名詞・一般に該当する単語をリストに格納
    words = []
    for parsed_word in parsed_words:
        if (    parsed_word[0] not in ('EOS', '') 
            and parsed_word[1] == '名詞'
            and ((parsed_word[2] == '固有名詞') or (parsed_word[2]=="一般") or (parsed_word[2]=="サ変接続"))):
            words.append(parsed_word[0])
    return words
""" 出演者データを拾って、上位100位の言葉を出力する。 """
def recommend_keywords(performer_words,performer):
    # 出演者の単語リストを取得
    words = performer_words.get(performer, [])
    
    # 単語の頻度を集計
    word_counts = defaultdict(int)
    for word in words:
        word_counts[word] += 1
    
    # 頻度順にソートして上位5つの単語を返す
    recommended = sorted(word_counts.items(), key=lambda x: x[1], reverse=True)[:100]
    
    return recommended
    #return [word for word, _ in recommended],recommended


## 出演者データの役職部分

In [48]:
#データから出演者データのみを切り取り
data["出演者1"]=data["出演者1"].apply(speakerTakeOut)
data["出演者2"]=data["出演者2"].apply(speakerTakeOut)
data["出演者3"]=data["出演者3"].apply(speakerTakeOut)
data["MC"]=data["MC"].apply(speakerTakeOut)

## タイトル、出演者データの整理

In [49]:
#データを辞書型にして「MC,タイトル」「出演者,タイトル」の辞書型データにしてリスト化
podcasts=[]
for title,p1,p2,p3,mc in zip(data["タイトル"],data["出演者1"],data["出演者2"],data["出演者3"],data["MC"]):
    podDict(podcasts,title,mc)
    if (pd.isna(p3)==False):
        podDict(podcasts,title,p1)
        podDict(podcasts,title,p2)
        podDict(podcasts,title,p3)
    elif (pd.isna(p2==False)):
        podDict(podcasts,title,p1)
        podDict(podcasts,title,p2)
    else:
        podDict(podcasts,title,p1)
podcasts[:3]

[{'title': '#1 ニューヨーク\u3000感染爆発、でも9割が対応支持\u3000NY覆う「戦時の空気」',
  'performer': '神田大介'},
 {'title': '#1 ニューヨーク\u3000感染爆発、でも9割が対応支持\u3000NY覆う「戦時の空気」',
  'performer': '藤原学思'},
 {'title': '#2 ローマ\u3000奔放イタリア、コロナで見せた従順\u3000マフィアのボスも…', 'performer': '神田大介'}]

## collectWord関数(タイトルの名詞を拾う関数)適用と辞書データに格納

In [50]:
performer_words = defaultdict(list)
for podcast in podcasts:
        # 単語を出演者データに追加
    performer_words[podcast["performer"]].extend(collectWords(podcast["title"]))
performer_words

defaultdict(list,
            {'神田大介': ['ニューヨーク',
              '感染',
              '爆発',
              '対応',
              '支持',
              'NY',
              '戦時',
              '空気',
              'ローマ',
              'イタリア',
              'コロナ',
              '従順',
              'マフィア',
              'ボス',
              'モスクワ',
              '医師',
              '警察',
              'ロシア',
              'プーチン',
              'モスクワ',
              '事態',
              '宣言',
              'プーチン',
              '大統領',
              '抗議',
              'バーチャル',
              'モスクワ',
              '欧米',
              '制限',
              'ロシア',
              '日本',
              'シンガポール',
              'ロボット',
              '管理',
              '社会',
              '理詰め',
              'コロナ',
              'シンガポール',
              '沈黙',
              '飯場',
              'コロナ',
              '拡散',
              '政府',
              'SIM',
              'カード',
              'シンガポール',
         

## 出演者データから頻出ワードを抽出

In [59]:
# 出演者名を入力しておすすめの検索ワードを出力
seek="向平真"
recommend_keywords(performer_words,seek)

[('前編', 9),
 ('後編', 9),
 ('謎', 6),
 ('発見', 6),
 ('平等院', 5),
 ('映画', 4),
 ('時代', 4),
 ('入門', 4),
 ('家康', 4),
 ('写真', 3),
 ('復元', 3),
 ('弥生', 3),
 ('本', 2),
 ('甲子園', 2),
 ('幻', 2),
 ('姿', 2),
 ('鉄', 2),
 ('傘', 2),
 ('寺', 2),
 ('興福', 2),
 ('寺東', 2),
 ('金堂', 2),
 ('倍', 2),
 ('出土', 2),
 ('薬師寺', 2),
 ('オーパーツ', 2),
 ('検閲', 2),
 ('フィルム', 2),
 ('断片', 2),
 ('葛藤', 2),
 ('大阪', 2),
 ('平城', 2),
 ('宮', 2),
 ('古代', 2),
 ('銅鐸', 2),
 ('鳳凰堂', 2),
 ('重文', 2),
 ('高槻', 2),
 ('硯', 2),
 ('文字', 2),
 ('石清水八幡宮', 2),
 ('ケヅレーライス', 1),
 ('レシピ', 1),
 ('大戦', 1),
 ('直前', 1),
 ('味', 1),
 ('由来', 1),
 ('敵国', 1),
 ('イギリス', 1),
 ('オイル', 1),
 ('装丁', 1),
 ('限界', 1),
 ('松本', 1),
 ('工房', 1),
 ('学校', 1),
 ('ルール', 1),
 ('欠席', 1),
 ('連絡', 1),
 ('ケヅレー', 1),
 ('おじさん', 1),
 ('おもちゃ', 1),
 ('日', 1),
 ('中', 1),
 ('戦争', 1),
 ('ウクライナ', 1),
 ('侵攻', 1),
 ('アルプス', 1),
 ('屋根', 1),
 ('一家', 1),
 ('情報', 1),
 ('価値', 1),
 ('かたち', 1),
 ('回廊', 1),
 ('日本', 1),
 ('帆走', 1),
 ('大空', 1),
 ('女性', 1),
 ('場面', 1),
 ('発掘', 1),
 ('作業', 1),
 ('沼', 1),
 ('記者',

# 参考出演者出演回数ソート

In [53]:
performer_counts = defaultdict(int)
for podcast in podcasts:
    performer_counts[podcast["performer"]] += 1
sorted(performer_counts.items(), key=lambda x: x[1], reverse=True)[:100]


[('神田大介', 882),
 ('岸上渉', 92),
 ('秋山訓子', 74),
 ('竹田真志夫', 65),
 ('安田桂子', 55),
 ('哲夫', 53),
 ('小寺右子', 51),
 ('桑原征平', 48),
 ('冨名腰隆', 45),
 ('桂吉弥', 42),
 ('向平真', 38),
 ('水野梓', 38),
 ('宮沢賢一', 37),
 ('仲程雄平', 33),
 ('杢田光', 30),
 ('真田嶺', 29),
 ('木下広大', 28),
 ('久保智祥', 26),
 ('中村俊介', 23),
 ('阿部朋美', 22),
 ('伊藤大地', 20),
 ('今永諒', 20),
 ('奥山晶二郎', 19),
 ('長沢美津子', 19),
 ('藤えりか', 18),
 ('乗京真知', 17),
 ('副島英樹', 17),
 ('飯沼優仁', 17),
 ('河原夏季', 15),
 ('宋光祐', 14),
 ('尾崎千裕', 14),
 ('河合真美江', 14),
 ('堀江麻友', 13),
 (None, 13),
 ('阪本輝昭', 13),
 ('須藤龍也', 12),
 ('太田匡彦', 11),
 ('藤田知也', 11),
 ('橋本佳奈', 11),
 ('藤原学思', 10),
 ('岡田玄', 10),
 ('竹園隆浩', 10),
 ('木村司', 10),
 ('Zeebra', 10),
 ('高橋健次郎', 9),
 ('加地ゆうき', 9),
 ('田部愛', 9),
 ('西村宏治', 8),
 ('星野典久', 8),
 ('吉岡桂子', 8),
 ('松本龍三郎', 8),
 ('伊藤あずさ', 8),
 ('星野眞三雄', 8),
 ('藤田直央', 7),
 ('小川詩織', 7),
 ('宮崎亮', 7),
 ('徳川家広', 7),
 ('清水唯一朗', 7),
 ('岡戸佑樹', 6),
 ('野島淳', 6),
 ('大久保真紀', 6),
 ('江渕崇', 6),
 ('杉浦奈実', 6),
 ('波絵理子', 6),
 ('池口龍法', 6),
 ('宮代栄一', 6),
 ('成沢解語', 6),
 ('小西良昭', 6),
 ('石橋亮介'